### import libraries

In [43]:
! pip install netCDF4

In [1]:
import netCDF4 # python API to work with netcdf (.nc) files
import os
import datetime
from osgeo import gdal, ogr, osr
import numpy as np # library to work with matrixes and computations in general
import matplotlib.pyplot as plt # plotting library
from auxiliary_classes import convert_time,convert_time_reverse,kelvin_to_celsius,kelvin_to_celsius_vector,Grid,Image,subImage
import json
import geojson, gdal, subprocess

### auxiliary functions

In [2]:
def print_geojson(tname, tvalue, fname, longitude, latitude, startdoc, position,endloop): #for printing to geojson - start,end,attributes
    fname = fname +".geojson"
    pmode="a"
    if startdoc==1:
        with open(fname, mode="w", encoding='utf-8') as f1: #start of geojson
            tstring = "{\n\"type\": \"FeatureCollection\",\n\"features\": ["
            print(tstring, file=f1)
            f1.close()
    else:
        if position==0: #for printing to geojson - geometry, longitude, latitude
            tstring = "\"type\": \"Feature\",\n\"geometry\": {\n\"type\": \"Point\",\n\"coordinates\": [" + str(longitude) + ","+ str(latitude) + "]\n},\n\"properties\": {"
            fname = fname 
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                print(tstring, file=f1)
                f1.close()
        elif position==1:  #start of point attributes
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                print("{", file=f1)
                f1.close()  
        elif position==2: #print attribute (not last)
             with open(fname, mode=pmode, encoding='utf-8') as f1:
                ttext = "\"" + str(tname) + "\": \"" +str(tvalue) + "\","
                print(ttext, file=f1) 
                f1.close() 
        elif position==3: #print last attribute
            with open(fname, mode=pmode, encoding='utf-8') as f1:
                ttext = "\"" + str(tname) + "\": \"" +str(tvalue) + "\""
                print(ttext, file=f1) 
                f1.close()        
        elif position==4: #end of point attributes
            with open(fname, mode=pmode, encoding='utf-8') as f1:  
                if endloop==0:
                    print("}\n},", file=f1)
                    f1.close()
                else:  #end of geojson
                    print("}\n}\n]\n}", file=f1)
                    f1.close()  

In [3]:
def trend(inputlist, nametrend, namediff, fname): 
    listlong = len(inputlist)
    if listlong <= 1:
        trendcoef = 0
        timediff = 0
        
    else:
        x = np.arange(0,len(inputlist))
        y = inputlist
        z = np.polyfit(x,y,1)
        trendcoef=z[0]
        timediff=int(trendcoef*(listlong-1))
    print_geojson(nametrend, trendcoef, fname, 0, 0, 0, 2, 0)
    print_geojson(namediff, timediff, fname, 0, 0, 0, 3, 0)
    
    
    

In [4]:
def trend2(inputlist, nametrend, namediff, endyear, startyear, fname,fnameavg):
    listlong = endyear-startyear+1
    numberweeks = len(inputlist[0])
    for j in range(0, numberweeks,1):
        tempweek = j +1
        if listlong <= 1:
            trendcoef = 0
            timediff = 0
            
        else:
            x = np.arange(0,listlong)
            y = []
            for i in range(0, listlong, 1): 
                y.append( inputlist[i][j])
            z = np.polyfit(x,y,1)
            trendcoef=z[0]
            timediff=int(trendcoef*(listlong-1))
        nametrend2 = nametrend + str(tempweek)
        namediff2 = namediff + str(tempweek)
        print_geojson(nametrend2, trendcoef, fname, 0, 0, 0, 2, 0)
        print_geojson(nametrend2, trendcoef, fnameavg, 0, 0, 0, 2, 0)
        if j == (numberweeks-1):
            print_geojson(namediff2, timediff, fname, 0, 0, 0, 3, 0)
            print_geojson(namediff2, timediff, fnameavg, 0, 0, 0, 3, 0)
        else:
            print_geojson(namediff2, timediff, fname, 0, 0, 0, 2, 0)
            print_geojson(namediff2, timediff, fnameavg, 0, 0, 0, 2, 0)
            
    
    

In [5]:
def avg2Dlist(inputlist,startyear,endyear): #average for 2D list ->1D list # inputs: inputlist = 2D list, output: avglist = 1D list with avg values
    numberyear = endyear-startyear+1
    listlen = len(inputlist[0])
    templist = []  
    avglist = []  
    for i in range(0, listlen,1):
        for j in range(0, numberyear,1):
            templist.append(inputlist[j][i])
        tempvalue=sum(templist)/len(templist)
        avglist.append(tempvalue) 
        templist = [] 
    return avglist
            

In [6]:
def acumulatelist(inputlist): #average for 2D list ->1D list # inputs: inputlist = 2D list, output: avglist = 1D list with avg values
    listlen = len(inputlist)
    for i in range (0,listlen-1,1):
        inputlist[i+1] += inputlist[i]
    return inputlist

In [7]:
def printlistasweekgeojson(inputlist,name,fname,fnameavg,endloop): # from list of week values print geojson
    listlen = len(inputlist)
    for i in range(0, listlen,1):
        tempvalue=inputlist[i]
        tvarname = name + str(i+1)
        if endloop==1 and i == (listlen-1):
            print_geojson(tvarname, tempvalue, fname, 0, 0, 0, 3, 0)
            print_geojson(tvarname, tempvalue, fnameavg, 0, 0, 0, 2, 0)
        else:
            print_geojson(tvarname, tempvalue, fname, 0, 0, 0, 2, 0)
            print_geojson(tvarname, tempvalue, fnameavg, 0, 0, 0, 2, 0)
        
    

###    evapotranspiration: function for one place

In [8]:
from datetime import date, timedelta
def findevapotranspiration(latitude,longitude,year,endyear,im,enddate, startdate, fnameevapot, allweekevapotlist,evapotranspirationparam, fnameannualprec, yearevapotlist, unitcoeff,fnameevapotaccum,fnamemevapotaccum, fnamemevapot,allmonthevapotlist):
    
    
    sdate = startdate   # start date for searching last frost date
    edate = enddate   # end date for searching last frost date
    delta = edate - sdate       # as timedelta
    sevendays=0 # for determination of new week (1-7)
    currentweek=1 # for determination of weeks
    weekevapotlist = []
    weekevapotranspiration=0
    weekevapotlist = []
    monthevapotlist =[]
    starthourday = 0
    endhourday = 23
    weekevapotsum = 0
    yearevapotsum = 0
    monthevapotsum = 0
    sdaylong = str(sdate)
    tmonth = int(sdaylong[5:7])
    currentmonth = tmonth
    
    for i in range(delta.days+1):
        daylong = sdate + timedelta(days=i)
        sdaylong = str(daylong)
        tday = int(sdaylong[8:10])
        tmonth = int(sdaylong[5:7])
        tyear = int(sdaylong[0:4])
        dayevapotsum = 0 # start value
        sevendays+=1
        for hour in range(starthourday, endhourday+1, 1): # for specific hours (all day,only sunrise hours,..)
            time=convert_time_reverse(datetime.datetime(tyear, tmonth, tday, hour, 0)) 
            slice_dictionary={'lon':[longitude,],'lat':[latitude],'time':[int(time)]}
            currentevapot=float(im.slice(evapotranspirationparam,slice_dictionary))*unitcoeff 
            dayevapotsum += currentevapot
            yearevapotsum += currentevapot
                    
        if daylong == edate: # save month date for last date in season
            monthevapotsum+=dayevapotsum
            monthevapotlist.append(monthevapotsum)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, monthevapotsum, fnamemevapot, 0, 0, 0, 2, 0)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, yearevapotsum, fnamemevapotaccum, 0, 0, 0, 2, 0)
        
        
        elif tmonth == currentmonth:
            monthevapotsum+=dayevapotsum
        
        else:
            monthevapotlist.append(monthevapotsum)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, monthevapotsum, fnamemevapot, 0, 0, 0, 2, 0)
            tvarname = "M" + str(year) + "_" + str(tmonth)
            print_geojson(tvarname, yearevapotsum, fnamemevapotaccum, 0, 0, 0, 2, 0)
            monthevapotsum=dayevapotsum
            currentmonth=tmonth
            
                
        if daylong == edate: # save week date for last date in season
            
            weekevapotsum+=dayevapotsum
            weekevapotlist.append(weekevapotsum)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekevapotsum, fnameevapot, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearevapotsum, fnameevapotaccum, 0, 0, 0, 2, 0)
            
        elif sevendays<=7:  # new week?   
            weekevapotsum+=dayevapotsum
            
        else:
            weekevapotlist.append(weekevapotsum)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, weekevapotsum, fnameevapot, 0, 0, 0, 2, 0)
            tvarname = "W" + str(year) + "_" + str(currentweek)
            print_geojson(tvarname, yearevapotsum, fnameevapotaccum, 0, 0, 0, 2, 0)
            weekevapotsum=dayevapotsum
            sevendays=0
            currentweek+=1
        
                    
          
    allweekevapotlist.append(weekevapotlist)
    allmonthevapotlist.append(monthevapotlist)
    yearevapotlist.append(yearevapotsum)
    tvarname = "Pr" + str(year) 
    print_geojson(tvarname, yearevapotsum, fnameannualprec, 0, 0, 0, 2, 0)
   
       
    
    
    
    
                  

### Find  evapotranspiration: function for selected years

In [9]:
def evapotranspirationyearly(latorder,lonorder,startyear,endyear,endloop,datafolder,fnameevapot,enddatem, startdatem,enddated, startdated,evapotranspirationparam, fnameannualprec, unitcoeff,fnameevapotaccum,fnameavgevapotranspiration,fnamemevapotaccum, fnamemevapot,fnameavgmevapotranspiration):
    print_geojson("", "", fnameevapot, 0, 0, 0, 1,0)
    print_geojson("", "", fnameevapotaccum, 0, 0, 0, 1,0)
    print_geojson("", "", fnameannualprec, 0, 0, 0, 1,0)
    print_geojson("", "", fnameavgevapotranspiration, 0, 0, 0, 1,0)
    print_geojson("", "", fnamemevapot, 0, 0, 0, 1,0)
    print_geojson("", "", fnamemevapotaccum, 0, 0, 0, 1,0)
    print_geojson("", "", fnameavgmevapotranspiration, 0, 0, 0, 1,0)
    endloopyear =0
   
    
    
    
    allweekevapotlist=[] # 2D list for all weeks many years
    allmonthevapotlist=[] # 2D list for all months many years
    yearevapotlist=[] # 2D list for all weeks many years
    for year in range(startyear, endyear+1, 1):
        source = datafolder + '/' + str(year) + '.nc' 
        im=Image(netCDF4.Dataset(source,'r'))   
        longlist = im.get_data().variables['lon'][:]
        latlist= im.get_data().variables['lat'][:]
        longitude = longlist [lonorder]   
        latitude = latlist[latorder]
        if year == startyear:
            print_geojson("", "", fnameevapot, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameannualprec, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameevapotaccum, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameavgevapotranspiration, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamemevapot, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnamemevapotaccum, longitude, latitude, 0, 0,0)
            print_geojson("", "", fnameavgmevapotranspiration, longitude, latitude, 0, 0,0)
        if year == endyear:
            endloopyear=1
            
            
        
        enddate=date(year, enddatem, enddated) 
        startdate=date(year, startdatem, startdated) 
                  
        
        
        
                 
            
        findevapotranspiration(latitude,longitude,year,endyear,im,enddate, startdate, fnameevapot, allweekevapotlist,evapotranspirationparam, fnameannualprec, yearevapotlist, unitcoeff,fnameevapotaccum,fnamemevapotaccum, fnamemevapot,allmonthevapotlist)
    
    
     
    
    
    avgweekevapotlist = avg2Dlist(allweekevapotlist,startyear,endyear)
    avgmonthevapotlist = avg2Dlist(allmonthevapotlist,startyear,endyear)
    printlistasweekgeojson(avgweekevapotlist,"PrW",fnameevapot,fnameavgevapotranspiration, 0)
    printlistasweekgeojson(avgmonthevapotlist,"PrM",fnamemevapot,fnameavgmevapotranspiration, 0)
    
    avgweekacuevapotlist = acumulatelist(avgweekevapotlist)
    avgmonthacuevapotlist = acumulatelist(avgmonthevapotlist)
    
    printlistasweekgeojson(avgweekacuevapotlist,"APrW",fnameevapotaccum,fnameavgevapotranspiration, endloopyear)
    printlistasweekgeojson(avgmonthacuevapotlist,"APrM",fnamemevapotaccum,fnameavgmevapotranspiration, endloopyear)
    
    
    avgevapotyear=sum(yearevapotlist)/len(yearevapotlist)
    print_geojson("AvgPre", avgevapotyear, fnameannualprec, 0, 0, 0, 2, 0)
    
    nametrend = "AnTrCo"
    namediff = "Andiff"
    trend(yearevapotlist, nametrend, namediff,fnameannualprec)
    
    nametrend = "TrCo"
    namediff = "Diff"
    trend2(allweekevapotlist, nametrend, namediff, endyear, startyear, fnameevapot,fnameavgevapotranspiration)
    trend2(allmonthevapotlist, nametrend, namediff, endyear, startyear, fnamemevapot,fnameavgmevapotranspiration)
   
    print_geojson("", "", fnameevapot, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnameannualprec, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnameevapotaccum, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameavgevapotranspiration, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnamemevapot, 0, 0, 0, 4,endloop)    
    print_geojson("", "", fnamemevapotaccum, 0, 0, 0, 4,endloop)
    print_geojson("", "", fnameavgmevapotranspiration, 0, 0, 0, 4,endloop)
          

### Find  evapotranspiration: function for selected latitudes, longitudes

In [10]:
def evapotranspirationplaces(startlat, startlon, endlat, endlon, startyear,endyear,exportfolder,datafolder,fnameevapot1,enddatem, startdatem,enddated, startdated, alllatlonfile, evapotranspirationparam, fnameannualprec1, unitcoeff,fnameevapotaccum1,fnameavgevapotranspiration1,fnamemevapotaccum1, fnamemevapot1,fnameavgmevapotranspiration1):
        fnameevapot= exportfolder + "/" +fnameevapot1
        fnameevapotaccum= exportfolder + "/" +fnameevapotaccum1
        fnameannualprec= exportfolder + "/" +fnameannualprec1
        fnameavgevapotranspiration= exportfolder + "/" +fnameavgevapotranspiration1
        fnamemevapot= exportfolder + "/" +fnamemevapot1
        fnamemevapotaccum= exportfolder + "/" +fnamemevapotaccum1
        fnameavgmevapotranspiration= exportfolder + "/" +fnameavgmevapotranspiration1
        
         #start in geojson files:
        print_geojson("", "", fnameevapot, 0, 0, 1, 0,0)
        print_geojson("", "", fnameevapotaccum, 0, 0, 1, 0,0)
        print_geojson("", "", fnameannualprec, 0, 0, 1, 0,0)
        print_geojson("", "", fnameavgevapotranspiration, 0, 0, 1, 0,0)
        print_geojson("", "", fnamemevapot, 0, 0, 1, 0,0)
        print_geojson("", "", fnamemevapotaccum, 0, 0, 1, 0,0)
        print_geojson("", "", fnameavgmevapotranspiration, 0, 0, 1, 0,0)
        
                
        endloop=0
        
        if alllatlonfile==1:  # if it is calculated for all latitudes and longitudes in input file
            source = datafolder + '/' + str(startyear) + '.nc' 
            im=Image(netCDF4.Dataset(source,'r')) 
            arraylon = im.get_data().variables['lon'][0::]
            arraylat = im.get_data().variables['lat'][0::]
            startlat=0
            startlon=0
            endlon= len(arraylon)-1
            endlat= len(arraylat)-1
        
        for latorder in range(startlat, endlat+1, 1):
            for lonorder in range(startlon, endlon+1, 1):
                if latorder==endlat and lonorder==endlon:
                    endloop=1
                evapotranspirationyearly(latorder,lonorder,startyear,endyear,endloop,datafolder,fnameevapot,enddatem, startdatem,enddated, startdated,evapotranspirationparam, fnameannualprec, unitcoeff,fnameevapotaccum,fnameavgevapotranspiration,fnamemevapotaccum, fnamemevapot,fnameavgmevapotranspiration)
       
       
        

## <font color=red>Find   evapotranspiration: input parameters and launch</font> 

In [ ]:
#Time definition:
startyear=2010 #start year (integer) 
endyear=2019 #end year (integer)  
enddatem = 12 # start date (month) each year
enddated = 31  # start date (day) each year
startdatem = 1 # end date (month) each year
startdated = 1 # end date (day) each year

#Optimalization:
starthourday=0 # integer 0-23
endhourday=23 # integer 0-23

#  unit:
units = 3  # 1 = m (default), 2 = cm, 3 = mm

#Files/Folders name:
datafolder = "data" #folder with data files (named by year) for each year #string
fnameevapot ="weekly_evapotranspiration" #name of created files with week evapotranspiration #string
fnameevapotaccum ="weekly_accum_evapotranspiration" #name of created files with week evapotranspiration #string
fnamemevapot ="monthly_evapotranspiration" #name of created files with month evapotranspiration #string
fnamemevapotaccum ="monthly_accum_evapotranspiration" #name of created files with month evapotranspiration #string
fnameavgevapotranspiration ="weekly_avg_evapotranspiration" #name of created files with week evapotranspiration #string
fnameavgmevapotranspiration ="monthly_avg_evapotranspiration" #name of created files with month evapotranspiration #string
fnameannualprec ="annualsum_evapotranspiration" #name of created files with annual/seasonal/defined period evapotranspiration #string
exportfolder = "export-evapo" #for all files (if each file its folder -> insert name of folder to name of file) #export folder must be created #string

#Area definition:
alllatlonfile=0 #calculate all latitudes and longitudes in input file (1=yes, 0=no)
# if alllatlonfile!=0 then:
startlat=4 # start number of list of latitudes from used netCDF4 file 
startlon=4 # start number of list of longitudes from used netCDF4 file 
endlat=17 # end number of list of latitudes from used netCDF4 file 
endlon=19 # end number of list of longitudes from used netCDF4 file 

#  evapotranspiration data parameter:
evapotranspirationparam = 'eow_lwe'

#  units coefficients 
unitcoeff = 1000
if units == 2:
    unitcoeff = 100000
elif units == 3: 
    unitcoeff = 1000000




evapotranspirationplaces(startlat, startlon, endlat, endlon, startyear,endyear,exportfolder,datafolder,fnameevapot,enddatem, startdatem,enddated, startdated,alllatlonfile,evapotranspirationparam, fnameannualprec, unitcoeff, fnameevapotaccum,fnameavgevapotranspiration,fnamemevapotaccum, fnamemevapot,fnameavgmevapotranspiration)






## From geojson to shp

In [25]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/weekly_evapotranspiration.shp', 'export/weekly_evapotranspiration.geojson']
subprocess.Popen(args)

In [26]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/weekly_accum_evapotranspiration.shp', 'export/weekly_accum_evapotranspiration.geojson']
subprocess.Popen(args)

In [23]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/annualsum_evapotranspiration.shp', 'export/annualsum_evapotranspiration.geojson']
subprocess.Popen(args)

In [ ]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/weekly_avg_evapotranspiration.shp', 'export/weekly_avg_evapotranspiration.geojson']
subprocess.Popen(args)

In [ ]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/monthly_evapotranspiration.shp', 'export/monthly_evapotranspiration.geojson']
subprocess.Popen(args)

In [ ]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/monthly_accum_evapotranspiration.shp', 'export/monthly_accum_evapotranspiration.geojson']
subprocess.Popen(args)

In [ ]:
args = ['ogr2ogr', '-f', 'ESRI Shapefile', 'export/shp/monthly_avg_evapotranspiration.shp', 'export/monthly_avg_evapotranspiration.geojson']
subprocess.Popen(args)